# Cryptopunks: obtener datos de un cryptopunk

In [1]:
import pandas as pd
import numpy as np


import requests
from bs4 import BeautifulSoup

import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.common.keys import Keys
from scrapy import Selector
import math

import mysql.connector

## Establecer conexión con la BD

In [2]:
# Connect to the new database
mydb = mysql.connector.connect(
  host="localhost",
  user="cecilia",
  password="cecilia",
  database="cryptopunks"
)

mycursor = mydb.cursor()



## Acceso a la página de un cryptopunk

In [15]:
# Define the cryptopunt to scrap
#nro_cp = 8348
nro_cp = 5214

url = 'https://www.larvalabs.com/cryptopunks/details/'+str(nro_cp)



In [16]:
# Use BeautifulSoup por webscrapping
html=requests.get(url)
soup = BeautifulSoup(html.content, 'html.parser')
#print(soup)

## Obtener el tipo del cryptopunk

In [17]:
h4 = soup.find_all('h4')
tipo_cp = h4[0].text.split()[3]
print(tipo_cp)

Male


## Obtener la lista de transacciones realizadas

In [18]:
text1 = soup.find_all('a', text = '')
#print(text1)

In [19]:
transacciones =[]

In [20]:
transac =soup.find('table', class_="table")
#print(transac)

for tr in transac.find_all('tr'):
    lista = []
    for td in tr.find_all('td'):
        lista.append(td.text.strip())
    transacciones.append(lista)
transacciones = transacciones[1:]
print(transacciones)

[['Bid Withdrawn', '0x6ad931', '', '95Ξ ($226,581)', 'Jan 24, 2022'], ['Bid', '0x6ad931', '', '95Ξ ($226,510)', 'Jan 23, 2022'], ['Bid Withdrawn', '0x6ad931', '', '87Ξ ($207,436)', 'Jan 23, 2022'], ['Bid', '0x6ad931', '', '87Ξ ($207,436)', 'Jan 23, 2022'], ['Bid Withdrawn', 'punksotc…', '', '75Ξ ($277,358)', 'Dec 31, 2021'], ['Offered', '', '', '102.69Ξ ($379,759)', 'Dec 31, 2021'], ['Bid', 'punksotc…', '', '75Ξ ($277,358)', 'Dec 31, 2021'], ['Bid', 'thebinoc…', '', '69Ξ ($255,170)', 'Dec 31, 2021'], ['Bid Withdrawn', 'bundo.et…', '', '21Ξ ($43,351)', 'Jul 23, 2021'], ['Bid', 'bundo.et…', '', '21Ξ ($39,573)', 'Jul 17, 2021'], ['Claimed', '', '0xcc7c33', '', 'Jun 23, 2017']]


In [21]:
# Insert data into the table
sql = "INSERT INTO transactions (cpID, tr_type, tr_from, tr_to, amount, txn) VALUES (%s, %s, %s, %s, %s, %s)"

for tr in transacciones:
    if len(tr)>0:
        mycursor.execute(sql, (nro_cp, tr[0],  tr[1], tr[2], tr[3], tr[4]))

mydb.commit()

In [22]:
## Obtener el nombre del owner, y saber si está a la venta y su precio

In [23]:
varios =soup.find_all('div', class_="col-md-10 col-md-offset-1")

for linea in varios:
    texto = linea.text
    # Busco la línea que comienza con ...
    if texto.strip().startswith('Current Market Status') and texto.find('currently')>0:
        # Extraigo el nombre del dueño
        if texto.find("This punk is currently owned by address")>0:
            owner = texto.split('address ')[1].split('.',1)[0]
            print('Owner:',owner)
        
        # Averiguo si está en venta
        precio_venta = 0
        if texto.find("This punk is currently for sale by owner")>0:
            sale = True
            precio_venta = texto.split('by owner for ')[1].split('Ξ',1)[0]
            print('Precio de venta', precio_venta)
        else:
            sale = False # Dice 'This punk has not been listed for sale by its owner'
        print('For sale',sale)
        
        # Averiguo si hay ofertas
        oferta = 0
        if texto.find("There is a bid of")>0:
            bids = True
            oferta = texto.split('is a bid of ')[1].split('Ξ',1)[0]
            print('Valor ofrecido', oferta)
             
        else:
            bids = False  # Dice "There are currently no bids on this punk."
        print('Bids:',bids)



Owner: 0xcc7c33
Precio de venta 102.69
For sale True
Bids: False


In [24]:
# Insert cryptopunk data into the table
sql = "INSERT INTO cryptopunks (cpID, owner, type, for_sale, value_price, bids, bid_value) VALUES (%s, %s, %s, %s, %s, %s, %s )"

mycursor.execute(sql, (nro_cp, owner, tipo_cp, sale, precio_venta, bids, oferta ))

mydb.commit()

## Obtener atributos del cryptopunk

In [25]:
cod =soup.find_all('div', class_="col-md-4")
textos = []
for c in cod:
    textos.append(c.find('a').text)
    
atributos = textos[1:]
print('Atributos:',atributos)

Atributos: ['Silver Chain', 'Wild Hair', 'Eye Patch']


In [26]:
# Insert data into the table attributes
sql = "INSERT INTO cp_attributes (cpID, attribute) VALUES (%s, %s)"

for atr in atributos:
    mycursor.execute(sql, (nro_cp,atr))

mydb.commit()